In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import math
import numpy as np
import json
from copy import deepcopy
import bisect

from config import filename
print(filename)

../Graphs/smallComplex.json


In [2]:

def n2letter(n):
    '''0 to 'a', 1 to 'b', ... '''
    return chr(96+n)

def string2duration(string):
    ''' "01:50:19.3177493" to duration in seconds'''
    return 3600*int(string[:2]) + 60*int(string[3:5]) + int(string[6:8])   #Duration is int

In [3]:
def read_data(path):
    global task_count
    global tasks
    file = open(path)
    data = json.load(file)
    nodes = data['nodes']
    tasks = dict()
    for task_str, info in nodes.items():
        task = int(task_str)
        tasks[task] = {'Data' : string2duration(info['Data']), 'Dependencies' : info['Dependencies']}
    task_count = len(tasks)
    print("Data loaded successfully. Number of tasks: " + str(task_count))

n_cores = 3
read_data(filename)
tasks

Data loaded successfully. Number of tasks: 1750


{1689: {'Data': 6619, 'Dependencies': [1715]},
 1690: {'Data': 654, 'Dependencies': [1689]},
 1697: {'Data': 3889, 'Dependencies': [1689]},
 1698: {'Data': 4357, 'Dependencies': [1697]},
 1699: {'Data': 3900, 'Dependencies': [1697]},
 1700: {'Data': 7236, 'Dependencies': [1699, 3478, 3482, 3507, 3498]},
 1701: {'Data': 1565, 'Dependencies': [1699]},
 1702: {'Data': 1741, 'Dependencies': [2084]},
 1703: {'Data': 2260, 'Dependencies': [1714]},
 1704: {'Data': 1876, 'Dependencies': [1701]},
 1705: {'Data': 2806, 'Dependencies': [1702]},
 1706: {'Data': 6993, 'Dependencies': [1703]},
 1709: {'Data': 4464, 'Dependencies': [1701]},
 1711: {'Data': 2794, 'Dependencies': [1702]},
 1712: {'Data': 768, 'Dependencies': [1709]},
 1714: {'Data': 3564, 'Dependencies': [1705]},
 1715: {'Data': 4715, 'Dependencies': [1706]},
 1718: {'Data': 4834, 'Dependencies': [1711]},
 1719: {'Data': 6815, 'Dependencies': [1712]},
 1720: {'Data': 5865, 'Dependencies': [1706]},
 1721: {'Data': 7362, 'Dependencies': 

In [4]:
#Tasks to child tasks / Tasks to parents / Task is terminal / Task is inital
task2childs = {task : list() for task in tasks}
task2parents = {task : list() for task in tasks}
for task, info in tasks.items():
    #Add childs
    list_task_parents = info['Dependencies']
    for task_parent in list_task_parents:
        task2childs[task_parent].append(task)
    #Add parents
    task2parents[task] = tasks[task]['Dependencies']
    
def task_is_terminal(task: int):
    return len(task2childs[task]) == 0
def task_is_inital(task: int):
    return len(task2parents[task]) == 0

# print(task2childs)
# print(task2parents)

In [6]:
task2sbl = {}

def save_static_bottom_level(task : int):
    task_duration = tasks[task]["Data"]
    if task_is_terminal(task):
        sbl = task_duration
    else:
        list_sbl_child = list()
        for task_child in task2childs[task]:
            if task_child in task2sbl:
                sbl_child = task2sbl[task_child]
            else:
                sbl_child = save_static_bottom_level(task_child)
            list_sbl_child.append(sbl_child)
        sbl = max(list_sbl_child) + task_duration
                
    task2sbl[task] = sbl
    return sbl

for task in tasks:
    if task_is_inital(task):
        save_static_bottom_level(task)
        
# task2sbl



In [7]:
class Node():
    def __init__(self, parent = None, task_to_add = None, core_where_to_add = None, time_task_start = None):
        '''Create a Node object ie a partial scheduling
        parent = parent Node, None if root
        task_to_add : task added to the partial schedule
        core_where_to_add : core where to do task
        time_task_start : instant where the core will start computing the task
        '''
        self.tasks_graph = tasks
        
        if parent is None:
            self.parent = None
            self.tasks_done_time = dict()
            self.cores = {core_n : {"task" : -1, "task_end_time" : 0} for core_n in range(n_cores)}
                                  
            self.hist = ''  
            
        else:
            task_end_time = time_task_start + self.tasks_graph[task_to_add]['Data']
            
            self.parent = parent
            self.tasks_done_time = parent.tasks_done_time.copy()
            self.tasks_done_time[task_to_add] = task_end_time

            self.cores = parent.cores.copy()
            self.cores[core_where_to_add] = {"task" : task_to_add, "task_end_time" : task_end_time}
                
            self.hist = parent.hist + f"|Task {task_to_add} start at time {time_task_start} on core {core_where_to_add} "
                 
    def __repr__(self):
        string = '[' + ','.join([n2letter(task) for task in self.tasks_done_time]) + ']'
        string += ''.join([f"({core['task']} end at {core['task_end_time']})" for core in self.cores.values()])
        return string
            
    def is_terminal(self):
        '''Return whether a node is a full schedule'''
        return len(self.tasks_done_time) == task_count
    

    def __lt__(self, other):
        return self.f < other.f
    def succ(self):                     
        '''Create and return list of child node of self'''
        childs = list()
        
        #On regarde toutes les tâches qu'on va tenter de rajouter
        for task, info in self.tasks_graph.items():
            
            #On passe les taches déjà ajoutées
            if task in self.tasks_done_time: 
                continue
            
            #On ne garde que les taches dont toutes les dépendances ont été réalisées
            if not all([task_required in self.tasks_done_time for task_required in info['Dependencies']]): 
                continue
            
            #On calcul le temps ou toutes les dépendances de task seront terminés par les coeurs   
            time_all_tasks_done = max([0] + [self.tasks_done_time[task_required] for task_required in info['Dependencies']])
                                         
            for core_n, core in self.cores.items():
                #On ne commence à faire la task que lorsque toutes les dépendances sont calculées et que le core est disponible.
                time_core_available = core["task_end_time"]
                time_task_start = max(time_all_tasks_done, time_core_available)
                child = Node(parent = self, task_to_add=task, core_where_to_add=core_n, time_task_start=time_task_start)
                childs.append(child)
                
        return childs
        
    def cost(self, child_node):
        '''Return the cost of going from self to child_node, a child node of self
        '''
        g_self = max([core["task_end_time"] for core in self.cores.values()])
        g_child = max([core["task_end_time"] for core in child_node.cores.values()])
        cost = g_child - g_self
        return cost
    
    #Node-schedule method
    def __hash__(self):
        return self.compute_g()
        
    def __eq__(self, node):
        '''Return whether a node is equal to another. Two nodes are considered equal if they have completed the same tasks and if all their cores stop working at same time.
        '''
        return self.tasks_done_time == node.tasks_done_time and self.set_of_core() == node.set_of_core()
        
    def set_of_core(self):
        return set([(core["task"], core["task_end_time"]) for core in self.cores.values()])
    
    def compute_g(self):
        return max([core["task_end_time"] for core in self.cores.values()])
    
    def compute_f(self):
        return self.f
# print(a, b, c, d, e, f, sep = '\n')
# print()
# print(g, h, i, j, k, l, sep = '\n')

In [8]:
def h(node):
    '''Estimated remaining time of the node-schedule for reaching a terminal node.
    Must understimate true value.'''
    successor_tasks = list()
    for task, info in tasks.items():
        if task in node.tasks_done_time: #On passe les taches déjà ajoutées
            continue
        if not all([task_required in node.tasks_done_time for task_required in info['Dependencies']]):   #On ne garde que les taches dont toutes les dépendances ont été réalisées
            continue
        successor_tasks.append(task)
    if successor_tasks == []:
        return 0
    return max([task2sbl[task] for task in successor_tasks])

In [9]:
import time
import heapq

class A_star():
    def __init__(self, root):
        self.root = root

    def find_best_path(self, max_time = float('inf')):
        t0 = time.time()
        self.root.g = 0
        self.root.f = h(self.root)
        
        OPEN_QUEUE = []
        heapq.heappush(OPEN_QUEUE,self.root)       #Open queue, pile of most urgent node to be evaluated
        OPEN_DICT = {self.root: None}   #Open set, more efficient way to compute if a node is in the open list
        CLOSED_DICT = dict()             #Closed list, list of already explored node
        
        while OPEN_QUEUE:
            # if time.time() - t0 > max_time:
            #     print('Time out')
            #     return
            
            current = heapq.heappop(OPEN_QUEUE)
                
            if current.is_terminal():   #If we reach a final node, it is the optimal solution and we return it
                return current

            for child in current.succ():
                child.g = child.compute_g()
                
                if child in CLOSED_DICT:    #We pass the node already visited
                    continue
                if child in OPEN_DICT:      #We pass the node already waiting to be visited (note: in A* general, we need to recompute child.g and child.parent here, but for us child.g only depends on child (not on parent))
                    continue
                
                else:
                    # child.g = current.g + current.cost(child)     #General method. In our case, node.g does not depend on the path and the parent node
                    child.g = child.compute_g()
                    child.f = child.g + h(child)
                    heapq.heappush(OPEN_QUEUE,child)
            
        raise Exception("No path from root to a terminal node")
    
            
    def insert_in_open_list(self, node, OPEN):
        print("here")
        

        

In [10]:
import cProfile
import pstats

root = Node()
a_star = A_star(root = root)

with cProfile.Profile() as pr:
    final_node = a_star.find_best_path()

print(final_node)
stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.dump_stats(filename='profiling.prof')

KeyboardInterrupt: 

In [ ]:
final_node.hist

'|Task 1 start at time 0 on core 0 |Task 2 start at time 2843 on core 1 |Task 3 start at time 2843 on core 2 |Task 4 start at time 2843 on core 0 |Task 5 start at time 6499 on core 2 |Task 7 start at time 6499 on core 1 |Task 6 start at time 7009 on core 0 |Task 8 start at time 11564 on core 2 |Task 9 start at time 10377 on core 1 |Task 10 start at time 15160 on core 2 '

In [ ]:
tasks



{1: {'Data': 2843, 'Dependencies': []},
 2: {'Data': 3656, 'Dependencies': [1]},
 3: {'Data': 2741, 'Dependencies': [1]},
 4: {'Data': 4166, 'Dependencies': [1]},
 5: {'Data': 5065, 'Dependencies': [2]},
 6: {'Data': 5116, 'Dependencies': [4]},
 7: {'Data': 3878, 'Dependencies': [2]},
 8: {'Data': 3596, 'Dependencies': [5]},
 9: {'Data': 5252, 'Dependencies': [7]},
 10: {'Data': 2883, 'Dependencies': [8]}}